<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/AtomVisionExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example to run [AtomVision](https://github.com/usnistgov/atomvision)

# Get the repository and install

In [30]:
!rm -rf atomvision
import os
!git clone https://github.com/usnistgov/atomvision.git
os.chdir('atomvision')
!python setup.py develop

Cloning into 'atomvision'...
remote: Enumerating objects: 1125, done.
remote: Counting objects: 100% (1125/1125), done.
remote: Compressing objects: 100% (1106/1106), done.
remote: Total 1125 (delta 20), reused 1118 (delta 15), pack-reused 0
Receiving objects: 100% (1125/1125), 63.87 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (20/20), done.
running develop
running egg_info
creating atomvision.egg-info
writing atomvision.egg-info/PKG-INFO
writing dependency_links to atomvision.egg-info/dependency_links.txt
writing requirements to atomvision.egg-info/requires.txt
writing top-level names to atomvision.egg-info/top_level.txt
writing manifest file 'atomvision.egg-info/SOURCES.txt'
adding license file 'LICENSE.md'
writing manifest file 'atomvision.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/atomvision.egg-link (link to .)
atomvision 2021.10.11 is already the active version in easy-install.pth

Installed /content/atomvision
Processing dependenc

In [ ]:
#!pip install torchvision

# Train DenseNet pretrained model on JARVIS-DFT 2D materials' STEM images generated with convolution approximation

In [ ]:
!python atomvision/scripts/train_classifiers.py --model_name densenet --train_folder atomvision/data/classification/stem_jv2d/train_folder --test_folder atomvision/data/classification/stem_jv2d/test_folder



Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100% 110M/110M [00:02<00:00, 39.5MB/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Training Results - Epoch: 1  Avg accuracy: 49.48 Avg loss: 3.90
Validation Results - Epoch: 1  Avg accuracy: 50.23 Avg loss: 4.06
Training Results - Epoch: 2  Avg accuracy: 66.59 Avg loss: 1.04
Validation Results - Epoch: 2  Avg accuracy: 64.52 Avg loss: 1.07
Training Results - Epoch: 3  Avg accuracy: 68.68 Avg loss: 0.84
Validation Results - Epoch: 3  Avg accuracy: 66.82 Avg loss: 0.86
Training Results - Epoch: 4 